In [1]:
using Distributions, Plots

In [221]:
candidate_n = 10 # Number of validator candidates
validator_n = 5 # Number of validator slots
nominator_n = 10; # Number of nominator candidates

In [224]:
# Nominator preferences, determined by their stake, preferred return and validators
nomination_preference(ordered_candidates::Vector, subset::Vector) = rand(subset, rand(1:length(ordered_candidates)))

nomination_preference (generic function with 2 methods)

In [230]:
candidate_stake(candidate_n::Int) = sort(rand(Pareto(1, 1), candidate_n)) # Validators represented only by their stake, always ordered
nominator_stake(nominator_n::Int) = sort(rand(Pareto(1, 1), nominator_n)) # Nominator stakes
nominator_interest(nominator_n::Int) = rand(Beta(10, 100), nominator_n) # Desired nominator interests

nominator_interest (generic function with 2 methods)

In [231]:
# Total stake
total_stake(validators::Vector, nominations) = validators + sum(nominations, 2)[:]
inflation(stakes::Vector, rates::Vector) = (rates' * stakes)[1] / sum(stakes)
inflation(validators::Vector, nominations, rates::Vector) = inflation(total_stake(validators, nominations), rates)
# Threshold
threshold(validator_n::Int, candidates::Vector, nominations) = minimum(sort(sum(nominations, 2)[:]+candidates)[end-validator_n+1:end])

threshold (generic function with 2 methods)

In [244]:
function sample(candidate_n::Int, nominator_n::Int, validator_n::Int, strat::Function, sample_n::Int)
    infs = zeros(sample_n)
    ths = zeros(sample_n)
    for s in 1:sample_n
        nominators = nominator_stake(nominator_n)
        candidates = candidate_stake(candidate_n)
        interests = nominator_interest(nominator_n);
        rates, nominations = strat(validator_n, candidate_n, candidates, nominator_n, nominators, interests)
        stakes = total_stake(candidates, nominations)
        infs[s] = inflation(stakes, rates)
        ths[s] = threshold(validator_n, candidates, nominations)/sum(stakes)
    end
    (infs, ths)
end

sample (generic function with 1 method)

In [245]:
function brute_strat(validator_n::Int, candidate_n::Int, candidates::Vector, nominator_n::Int, nominators::Vector, interests::Vector)
    rates = rand(Beta(10, 100), candidate_n)
    nominations = zeros(candidate_n, nominator_n)
    subset = rand(1:candidate_n, validator_n)
    for n in 1:nominator_n
        preference = nomination_preference(candidates, subset)
        c = length(preference)
        for p in preference
            if rates[p] >= interests[n]
                nominations[p, n] += nominators[n]/c
            end
        end
    end
    (rates, nominations)
end

brute_strat (generic function with 2 methods)

In [249]:
function adarates_strat(validator_n::Int, candidate_n::Int, candidates::Vector, nominator_n::Int, nominators::Vector, interests::Vector)
    rates = ones(candidate_n)
    nominations = zeros(candidate_n, nominator_n)
    subset = rand(1:candidate_n, validator_n)
    for n in 1:nominator_n
        preference = nomination_preference(candidates, subset)
        c = length(preference)
        for p in preference
            if interests[n] < rates[p]
                rates[p] = interests[n]
            end
            nominations[p, n] += nominators[n]/c
        end
    end
    (rates, nominations)
end

adarates_strat (generic function with 1 method)

In [259]:
# X is inflation, Y is threshold
scatter(sample(candidate_n, nominator_n, validator_n, brute_strat, 1000))
scatter!(sample(candidate_n, nominator_n, validator_n, adarates_strat, 1000))

In [243]:
plot(rates)